In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from skl2onnx.helpers.onnx_helper import load_onnx_model
from stitchnet.stitchonnx.utils import Net
from glob import glob
from pathlib import Path

netsFiles = sorted(glob('_models/fragments/net*'))
nets = []
for i,netsFile in enumerate(netsFiles):
    fragmentFiles = sorted(glob(str(Path(netsFile)/'fragment*.onnx')))
    onnxFragments = []
    for fragmentFile in fragmentFiles:
        onnxFragment = load_onnx_model(fragmentFile)
        onnxFragments.append(onnxFragment)
    net1 = Net(onnxFragments, i)
    nets.append(net1)

/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from stitchnet.stitchonnx.utils import load_cats_and_dogs_dset,convert_imagenet_to_cat_dog_label
from stitchnet.stitchonnx.utils import accuracy_score_model,accuracy_score_net,load_dl
from stitchnet.stitchonnx.utils import generate_networks, ScoreMapper
from stitchnet.stitchonnx.report import Report
from stitchnet.stitchonnx.utils import evalulate_stitchnet

from tqdm import tqdm
import torch
import numpy as np
import os
from collections import defaultdict
import hashlib

STITCH_BATCH_SIZE = 64
EVAL_BATCH_SIZE = 64
MAX_DEPTH = 10
THRESOULD = 0.8
TOTAL_THRESOULD = 0.5

RESULT_NAME = "result"

dataset_train = load_cats_and_dogs_dset("train")
dataset_val = load_cats_and_dogs_dset("test")

dl_score = load_dl(dataset_train, STITCH_BATCH_SIZE)
data_score,t = next(iter(dl_score))
data_score = data_score.numpy()

In [4]:
# range(1)
k = 0
if os.path.exists(f'./_results/{RESULT_NAME}.txt'):
    with open(f'./_results/{RESULT_NAME}.txt', 'r') as f:
        k = len(f.read().split('\n'))

In [5]:
from tqdm import tqdm

scoreMapper = ScoreMapper(nets, data_score)
with Report(EVAL_BATCH_SIZE, f'./_results/{RESULT_NAME}.txt', 'a') as report:
    # for _ in tqdm(range(50)):
    generator = generate_networks(nets, scoreMapper, data_score, 
                          threshold=THRESOULD, totalThreshold=TOTAL_THRESOULD, 
                          maxDepth=MAX_DEPTH, sample=False)
    for i,(s,net) in enumerate(generator):
        netname = f"_results/{RESULT_NAME}/net{k:03}_{i:03}"
        report.evaluate(nets, net, netname, s, dataset_val)
        net.save(netname)
        k += 1

/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


current depth: 1


100%|██████████| 6/6 [00:26<00:00,  4.49s/it]


potential next fragments: 48
potential next fragments after thresholding of 0.8: 2 ['1.0', '0.95']
totalscore 0.9999998807907104
current depth: 2


100%|██████████| 6/6 [00:06<00:00,  1.09s/it]


...

  0%|          | 0/2 [00:00<?, ?it/s]/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
100%|██████████| 2/2 [00:04<00:00,  2.13s/it]


totalscore 0.8250401407527856


  0%|          | 0/2 [00:00<?, ?it/s]/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
100%|██████████| 2/2 [00:04<00:00,  2.29s/it]


totalscore 0.8125636354164412


  0%|          | 0/2 [00:00<?, ?it/s]/opt/anaconda3/envs/stitchnet/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:54: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(
100%|██████████| 2/2 [00:04<00:00,  2.15s/it]


In [6]:
# import graphviz

# ni = graphviz.Graph('ni')  

# ni.attr('node', shape='rarrow')
# ni.node('1', 'Ni!')
# ni.node('2', 'Ni!')

# ni.node('3', 'Ni!', shape='egg')

# ni.attr('node', shape='star')
# ni.node('4', 'Ni!')
# ni.node('5', 'Ni!')
# ni.format = 'svg'
# ni.render(f'test', view=False)

In [3]:
# !dot

In [4]:
# import os
# print(os.environ['PATH'])